In [5]:
%pip install -r requirements.txt # for intial use, please run this line to install the required packages

In [ ]:
#Library Import

import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import altair as alt
import scipy.stats as stats
from scipy.stats import norm

# import seaborn as sns
# from sklearn.neighbors import KernelDensity
# from bokeh.models import ColumnDataSource, Label, PrintfTickFormatter
# from bokeh.palettes import Dark2_5 as colors
# from bokeh.plotting import figure, show
# from bokeh.sampledata.cows import data as df
# from bokeh.io import output_notebook


In [11]:
#copied from main.ipynb
# #import pandas as pd
df_course = pd.read_csv('./data/student.course.csv')
df_record = pd.read_csv('./data/student.record.csv')
df_term = pd.read_table('./data/term.table.txt', delimiter="\t").fillna('Unknown')

df_term.columns = ['TERM_ID', 'TERM_DESCRIPTION']

terms = ['Fall', 'Winter', 'Summer', 'Spring', 'Unknown']
df_term['TERM_NAME'] = df_term['TERM_DESCRIPTION'].apply(lambda x: list(filter(lambda y: y!='', [t if t in x else '' for t in terms]))[0])
df_term['TERM_YEAR'] = df_term['TERM_DESCRIPTION'].apply(lambda x: x[-4:] if x[-4:].isnumeric() else None)

merge_course_df = pd.merge(
                    df_course,
                    df_term, 
                    how='left',
                    left_on='TERM',
                    right_on='TERM_ID')

terms_student = ['ADMIT_TERM', 'MAJOR1_TERM', 'MAJOR2_TERM', 'MAJOR3_TERM']
merge_record_df = df_record.copy()
for term in terms_student:
    
    merge_record_df = pd.merge(
                        merge_record_df,
                        df_term, 
                        how='left',
                        left_on=term,
                        right_on='TERM_ID')
    
    

    merge_record_df.rename(columns={'TERM_ID': term + '_' + 'TERM_ID', 
                            'TERM_DESCRIPTION': term + '_' + 'TERM_DESCRIPTION', 
                            'TERM_NAME': term + '_' + 'TERM_NAME', 
                            'TERM_YEAR': term + '_' + 'TERM_YEAR'}, inplace=True)    
    
 


# df_term
# df_course
# df_record
# merge_course_df
merge_record_df

    
course_columns = ['ANONID', 'SUBJECT', 'CATALOG_NBR', 'GRD_PTS_PER_UNIT', 'GPAO', 'DIV',
                  'ANON_INSTR_ID', 'TERM', 'TERM_ID', 'TERM_DESCRIPTION', 'TERM_NAME', 'TERM_YEAR']

record_columns = ['MAJOR3_DESCR', 'MAJOR2_DESCR', 'MAJOR1_DESCR', 'HSGPA', 'LAST_ACT_ENGL_SCORE', 'LAST_ACT_MATH_SCORE',
                  'LAST_ACT_READ_SCORE', 'LAST_ACT_SCIRE_SCORE', 'LAST_ACT_COMP_SCORE', 'LAST_SATI_VERB_SCORE', 
                  'LAST_SATI_MATH_SCORE', 'LAST_SATI_TOTAL_SCORE', 'SEX', 'STDNT_GROUP1', 'STDNT_GROUP2', 'MAJOR1_DEPT', 
                  'MAJOR2_DEPT', 'MAJOR3_DEPT', 'ANONID', 'ADMIT_TERM', 'MAJOR1_TERM', 'MAJOR2_TERM', 'MAJOR3_TERM', 
                  'MAJOR3_TERM_TERM_ID', 'MAJOR3_TERM_TERM_DESCRIPTION', 'MAJOR3_TERM_TERM_NAME', 'MAJOR3_TERM_TERM_YEAR']   

# merge_course_df.describe()
# merge_record_df.describe()


# https://www.bls.gov/oes/tables.htm
# https://data.census.gov/table/ACSST1Y2015.S0101
# https://data.census.gov/table/ACSST1Y2015.S1501?t=Education&g=040XX00US06
# merge_record_df.sort_values(by=['MAJOR3_TERM_TERM_YEAR'], ascending=[True])['MAJOR3_TERM_TERM_YEAR'].unique()
# 1992 - 2015


# Occupational data by gender
# https://www.bls.gov/cps/cps_aa2015.htm

# https://www.axios.com/2020/06/30/half-us-population-unemployed
# the labor market is only at 50%?


merge_record_df.head()


,MAJOR3_DESCR,MAJOR2_DESCR,MAJOR1_DESCR,HSGPA,LAST_ACT_ENGL_SCORE,LAST_ACT_MATH_SCORE,LAST_ACT_READ_SCORE,LAST_ACT_SCIRE_SCORE,LAST_ACT_COMP_SCORE,LAST_SATI_VERB_SCORE,...,MAJOR1_TERM_TERM_NAME,MAJOR1_TERM_TERM_YEAR,MAJOR2_TERM_TERM_ID,MAJOR2_TERM_TERM_DESCRIPTION,MAJOR2_TERM_TERM_NAME,MAJOR2_TERM_TERM_YEAR,MAJOR3_TERM_TERM_ID,MAJOR3_TERM_TERM_DESCRIPTION,MAJOR3_TERM_TERM_NAME,MAJOR3_TERM_TERM_YEAR
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Asian Studies BA,3.8,25.0,22.0,29.0,28.0,26.0,NaN,...,Winter,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Psychology BA,3.3,NaN,NaN,NaN,NaN,NaN,NaN,...,Winter,2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,Civil Engineering BSE,3.9,33.0,32.0,28.0,30.0,31.0,NaN,...,Summer,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,3.8,24.0,22.0,18.0,18.0,21.0,610.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
#from main.ipynb

import pandas as pd

# More tables here
# https://www.bls.gov/cps/tables.htm
# BLS MEDAIN WEEKLY EARNINGS https://www.bls.gov/cps/cpsaat39.htm

# https://data.bls.gov/apps/demographic-comparison-matrix/home.htm

# treemaps https://docs.bokeh.org/en/latest/docs/examples/topics/hierarchical/treemap.html

# https://data.census.gov/table/ACSST1Y2015.S1501?t=Education&g=040XX00US06

df_bls_2015_headers_1 = pd.read_excel('./data/bls_cpsaat09_2002_to_2015.xlsx', sheet_name='2015', header=0).fillna('Unknown').iloc[0]
df_bls_2015_headers_2 = pd.read_excel('./data/bls_cpsaat09_2002_to_2015.xlsx', sheet_name='2015', header=1).fillna('Unknown').iloc[0]
df_bls_2015_headers_3 = pd.read_excel('./data/bls_cpsaat09_2002_to_2015.xlsx', sheet_name='2015', header=2).fillna('Unknown').iloc[0]
df_bls_2015_headers_4 = pd.read_excel('./data/bls_cpsaat09_2002_to_2015.xlsx', sheet_name='2015', header=3).fillna('Unknown').iloc[0]

df_bls_2015 = pd.read_excel('./data/bls_cpsaat09_2002_to_2015.xlsx', sheet_name='2015', header=3).fillna('Unknown')

df_bls_2015

tabs = ['2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006', '2005', '2004', '2003']

columns = ['level', 
           'l3','l2','l1','l0',
           'occupation', 
           'total_16_years_and_over_py', 
           'total_16_years_and_over',
           'male_16_years_and_over_py', 
           'male_16_years_and_over',
           'male_20_years_and_over_py', 
           'male_20_years_and_over',
           'female_16_years_and_over_py', 
           'female_16_years_and_over',
           'female_20_years_and_over_py', 
           'female_20_years_and_over']

# combined = list(zip(df_bls_2015_headers_1, df_bls_2015_headers_2, df_bls_2015_headers_3, df_bls_2015_headers_4))
# for row in combined:
#     print(row)


df_bls_all = pd.DataFrame()
for tab in tabs:
    df_bls_next = pd.read_excel('./data/bls_cpsaat09_2002_to_2015.xlsx', 
                                sheet_name=str(tab), 
                                header=3, 
                                ).fillna('Unknown')
    
    df_bls_next.columns = columns
    df_bls_next['year'] = str(tab)
    # df_bls_all.columns = columns.append('year')
    df_bls_all = pd.concat([df_bls_all, df_bls_next])

#print(len(df_bls_all))
df_bls_all_filtered = df_bls_all[df_bls_all['occupation'] != 'Unknown']
df_bls_all_filtered_Totals = df_bls_all[(df_bls_all['level'] == 'L3') & (df_bls_all['total_16_years_and_over_py'] != 'Unknown')]
df_bls_all_filtered_L2 = df_bls_all[(df_bls_all['level'] == 'L2') & (df_bls_all['total_16_years_and_over_py'] != 'Unknown')]
df_bls_all_filtered_L0 = df_bls_all[(df_bls_all['level'] == 'L0') & (df_bls_all['total_16_years_and_over_py'] != 'Unknown')]


# Check L0 Sums vs Totals
# for tab in tabs:
#     print(tab, df_bls_all_filtered_L0[df_bls_all_filtered_L0['year'] == tab]['total_16_years_and_over_py'].sum(axis=0) - df_bls_all_filtered_Totals[df_bls_all_filtered_Totals['year'] == tab]['total_16_years_and_over_py'].sum(axis=0))


# print(len(df_bls_all_filtered_L0[(df_bls_all_filtered_L0['year'] == '2015')]))
# print(df_bls_all_filtered_L0[(df_bls_all_filtered_L0['year'] == '2015')].columns)
# print(df_bls_all_filtered_L0[(df_bls_all_filtered_L0['year'] == '2015')]['occupation'].unique())

# Sanity Check trend
#for tab in tabs:
 #   print(tab, df_bls_all_filtered_L0[(df_bls_all_filtered_L0['year'] == tab) & (df_bls_all_filtered_L0['occupation'] == 'Computer and mathematical occupations')]['total_16_years_and_over_py'].sum(axis=0))


df_bls_all_filtered


,level,l3,l2,l1,l0,occupation,total_16_years_and_over_py,total_16_years_and_over,male_16_years_and_over_py,male_16_years_and_over,male_20_years_and_over_py,male_20_years_and_over,female_16_years_and_over_py,female_16_years_and_over,female_20_years_and_over_py,female_20_years_and_over,year
0,L3,Unknown,Unknown,Unknown,Unknown,Total,146305.0,148834.0,77692.0,79131.0,75471.0,76776.0,68613.0,69703.0,66287.0,67323.0,2015
2,L2,Unknown,Unknown,Unknown,Unknown,"Management, professional, and related occupations",56050.0,57960.0,27119.0,28090.0,26976.0,27918.0,28931.0,29871.0,28751.0,29658.0,2015
3,L1,Unknown,Unknown,Unknown,Unknown,"Management, business, and financial operations...",23171.0,24108.0,13041.0,13589.0,13008.0,13548.0,10129.0,10519.0,10098.0,10476.0,2015
4,L0,Total,"Management, professional, and related occupations","Management, business, and financial operations...",Management occupations,Management occupations,16199.0,16994.0,9953.0,10340.0,9931.0,10309.0,6246.0,6654.0,6223.0,6624.0,2015
5,L0,Total,"Management, professional, and related occupations","Management, business, and financial operations...",Business and financial operations occupations,Business and financial operations occupations,6972.0,7114.0,3088.0,3249.0,3077.0,3238.0,3884.0,3866.0,3875.0,3851.0,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39,L0,Total,"Natural resources, construction, and maintenan...","Natural resources, construction, and maintenan...",Office and administrative support occupations,Construction and extraction occupations,7898.0,8114.0,7674.0,7891.0,7431.0,7636.0,224.0,223.0,215.0,214.0,2003
41,L0,Total,"Natural resources, construction, and maintenan...","Natural resources, construction, and maintenan...",Office and administrative support occupations,"Installation, maintenance, and repair occupations",4623.0,5041.0,4412.0,4830.0,4312.0,4730.0,212.0,211.0,206.0,204.0,2003
44,L2,Unknown,Unknown,Unknown,Unknown,"Production, transportation, and material movin...",18569.0,18020.0,14091.0,13745.0,13509.0,13248.0,4478.0,4274.0,4334.0,4143.0,2003
45,L0,Total,"Natural resources, construction, and maintenan...","Natural resources, construction, and maintenan...",Office and administrative support occupations,Production occupations,10081.0,9700.0,6863.0,6696.0,6682.0,6566.0,3218.0,3004.0,3144.0,2938.0,2003


In [15]:
def data_prep(data1, data2):
    # Student Data Prep
    major_df = pd.read_excel("./data/majors.xlsx")
    student_Occupations_df = pd.merge(
                    data1,
                    major_df, 
                    how='left',
                    left_on='MAJOR1_DESCR',
                    right_on='MAJOR')

    pivot_table = pd.pivot_table(
        student_Occupations_df,
        index=['OCCUPATION', 'MAJOR1_TERM_TERM_YEAR'],  # Rows
        columns='SEX',                                  # Columns
        aggfunc='size',                                 # Count of occurrences
        fill_value=0                                    # Fill missing values with 0
    )
    pivot_table.reset_index(inplace=True)
    pivot_table['MAJOR1_TERM_TERM_YEAR'] = pivot_table['MAJOR1_TERM_TERM_YEAR'].apply(lambda x: int(x))

    student_melted = pd.melt(pivot_table, 
                        id_vars=['MAJOR1_TERM_TERM_YEAR', 'OCCUPATION'],  # Columns to keep
                        value_vars=['F', 'M'],  # Columns to unpivot
                        var_name='SEX',  # Name for new column indicating Sex (Females/Males)
                        value_name='count')  # Name for new column holding the values

    # BLS Data Prep
    def getMales(row):
        x = row['male_16_years_and_over'] + row['male_20_years_and_over']
        return x if isinstance(x, float) else None

    data2['Males'] = data2.apply(lambda x: getMales(x), axis=1)

    def getFemales(row):
        x = row['female_16_years_and_over'] + row['female_20_years_and_over']
        return x if isinstance(x, float) else None

    data2['Females'] = data2.apply(lambda x: getFemales(x), axis=1)

    table = pd.pivot_table(data2, values=['Males', 'Females'], index=['year', 'occupation'],
                           aggfunc="sum")
    table.reset_index(inplace=True)
    table = table[table['occupation'] != 'Unknown']
    table['year'] = table['year'].apply(lambda x: int(x))

    # Fix the filter here, use `table['year']` not `pivot_table['year']`

    # Melt the BLS data
    bls_melted = pd.melt(table, 
                        id_vars=['year', 'occupation'],  # Columns to keep
                        value_vars=['Females', 'Males'],  # Columns to unpivot
                        var_name='SEX',  # Name for new column indicating Sex (Females/Males)
                        value_name='count')  # Name for new column holding the values
    
    student_melted = student_melted.rename(columns={'MAJOR1_TERM_TERM_YEAR': 'year'})
    bls_melted = bls_melted.rename(columns={'occupation': 'OCCUPATION'})

    return student_melted, bls_melted

# Call the function
student_melted, bls_melted = data_prep(merge_record_df, df_bls_all_filtered)


C:\Users\alasw\AppData\Local\Temp\ipykernel_35512\460418116.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Males'] = data2.apply(lambda x: getMales(x), axis=1)
C:\Users\alasw\AppData\Local\Temp\ipykernel_35512\460418116.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Females'] = data2.apply(lambda x: getFemales(x), axis=1)


,year,OCCUPATION,SEX,count
0,1982,Architecture and engineering occupations,F,0
1,1983,Architecture and engineering occupations,F,0
2,1984,Architecture and engineering occupations,F,2
3,1985,Architecture and engineering occupations,F,0
4,1986,Architecture and engineering occupations,F,2
...,...,...,...,...
579,2012,Office and administrative support occupations,M,251
580,2013,Office and administrative support occupations,M,276
581,2014,Office and administrative support occupations,M,301
582,2015,Office and administrative support occupations,M,224


In [20]:
def TimeSeriesBar(data, occupation, people):
    data = data.copy()
    data = data[(data['year'] > 2007) & (data['year'] <= 2016)]

    # Ensure the 'year' column is in the correct format (datetime)
    data['year'] = pd.to_datetime(data['year'].astype(str))
    data = data[data['OCCUPATION'] == occupation]

    # Define your custom color scheme
    custom_colors = [
        '#FBEA65',  # for Female
        '#081F49',  # for Male
    ]

    # Create the stacked bar chart with specified color order and no legend
    bar_chart = alt.Chart(data).mark_bar(size=15, stroke='black', strokeWidth=1).encode(
        x=alt.X("year:T",
                axis=alt.Axis(grid=False, title="Year"),
                scale=alt.Scale(domain=[pd.to_datetime('2007-01-01'), pd.to_datetime('2016-01-01')])
        ),
        y=alt.Y('count', stack=True, axis=alt.Axis(grid=False, title=people)),
        color=alt.Color("SEX", 
                        scale=alt.Scale(range=custom_colors),  # Use the custom colors for SEX
                        legend=None  # Remove the legend
        )
    ).properties(
        title={
            "text": f"{occupation}",
            "subtitle": f"Distribution by Sex for {people}",
            "anchor": "start"        # Align title to the left
        },
        width=200,    # Set custom width
        height=200   # Set custom height
    )
    
    return bar_chart





TimeSeriesBar(student_melted, 'Life, physical, and social science occupations', "Students")

alt.Chart(...)

In [21]:
TimeSeriesBar(bls_melted, 'Life, physical, and social science occupations', "Workers (in thousands)")

alt.Chart(...)

In [22]:
TimeSeriesBar(student_melted, 'Architecture and engineering occupations', "Students")


alt.Chart(...)

In [23]:
TimeSeriesBar(bls_melted, 'Architecture and engineering occupations', "Workers (in thousands)")

alt.Chart(...)

In [24]:
TimeSeriesBar(student_melted, 'Management occupations', "Students")

alt.Chart(...)

In [25]:
TimeSeriesBar(bls_melted, 'Management occupations', "Workers (in thousands)")

alt.Chart(...)

# Pivot tables
AUSTIN! How did you make this excel sheet?

In [19]:
data_to_be_pivoted = pd.read_excel("assets/df_merge_bls_l1_pivoted.xlsx", 
                                     sheet_name="df_merge_bls_l1", 
                                     header=0)
data_to_be_pivoted = data_to_be_pivoted[data_to_be_pivoted['year'] == 2015] 
data_to_be_pivoted.columns

Index(['year', 'l1', 'number_of_students_all_sum',
       'number_of_students_men_sum', 'number_of_students_women_sum',
       'number_of_students_unknown_sum', 'number_of_workers_all_sum',
       'median_weekly_earnings_all_mean', 'number_of_workers_men_sum',
       'median_weekly_earnings_men_mean', 'number_of_workers_women_sum',
       'median_weekly_earnings_women_mean'],
      dtype='object')

In [20]:
worker_total = data_to_be_pivoted['number_of_workers_all_sum'].sum()
student_total = data_to_be_pivoted['number_of_students_all_sum'].sum()

delta_table = pd.pivot_table(data_to_be_pivoted, values=['number_of_students_all_sum', 'number_of_workers_all_sum'], index='l1',
                       aggfunc="sum", fill_value=0)
delta_table = delta_table[delta_table['number_of_students_all_sum'] != 0]

delta_table['% students'] = delta_table['number_of_students_all_sum']/student_total
delta_table['% workers'] = delta_table['number_of_workers_all_sum']/worker_total

delta_table['delta'] = delta_table['% workers'] - delta_table['% students']


delta_table[(delta_table['delta'] >= 0.06) | (delta_table['delta'] <= -0.06)]


,number_of_students_all_sum,number_of_workers_all_sum,% students,% workers,delta
l1,,,,,
Architecture and engineering occupations,761.0,2655.0,0.150514,0.024339,-0.126175
"Life, physical, and social science occupations",2330.0,1174.0,0.460839,0.010762,-0.450076
Management occupations,58.0,12479.0,0.011472,0.114399,0.102928


In [42]:
student_table = pd.pivot_table(data_to_be_pivoted[(data_to_be_pivoted['l1'] == "Architecture and engineering occupations") |(data_to_be_pivoted['l1'] == "Life, physical, and social science occupations") |(data_to_be_pivoted['l1'] == "Management occupations")],
                                values=['number_of_students_women_sum', 'number_of_students_all_sum'], index='l1',
                       aggfunc="sum", fill_value=0)

student_table['%women'] = ((student_table['number_of_students_women_sum']/student_table['number_of_students_all_sum']) * 100).round(decimals=0)
student_table

,number_of_students_all_sum,number_of_students_women_sum,%women
l1,,,
Architecture and engineering occupations,761.0,211.0,28.0
"Life, physical, and social science occupations",2330.0,1366.0,59.0
Management occupations,58.0,31.0,53.0


In [43]:
workers_table = pd.pivot_table(data_to_be_pivoted[(data_to_be_pivoted['l1'] == "Architecture and engineering occupations") |(data_to_be_pivoted['l1'] == "Life, physical, and social science occupations") |(data_to_be_pivoted['l1'] == "Management occupations")],
                                values=['number_of_workers_women_sum', 'number_of_workers_all_sum'], index='l1',
                       aggfunc="sum", fill_value=0)

workers_table['%women'] = ((workers_table['number_of_workers_women_sum']/workers_table['number_of_workers_all_sum']) * 100).round(decimals=0)
workers_table

,number_of_workers_all_sum,number_of_workers_women_sum,%women
l1,,,
Architecture and engineering occupations,2655.0,382.0,14.0
"Life, physical, and social science occupations",1174.0,516.0,44.0
Management occupations,12479.0,5146.0,41.0


In [44]:

student_table.reset_index(inplace = True)
workers_table.reset_index(inplace = True)

In [50]:
#!pip install statsmodels
from statsmodels.stats.proportion import proportions_ztest



for l in student_table['l1']:
    X1 = student_table['number_of_students_women_sum'][student_table['l1'] == l].values[0]
    X2 = workers_table['number_of_workers_women_sum'][workers_table['l1'] == l].values[0]
    n1 = student_table['number_of_students_all_sum'][student_table['l1'] == l].values[0]
    n2 = workers_table['number_of_workers_all_sum'][workers_table['l1'] == l].values[0]

    # Number of successes in both groups
    count = [X1, X2]

    # Sample sizes for both groups
    nobs = [n1, n2]

    # Perform the two-proportion z-test
    stat, p_value = proportions_ztest(count, nobs)

    # Round the Z-statistic to 2 decimal places
    stat_rounded = round(stat, 2)

    # Format p_value to 3 decimal places, or print '< .001' if it's less than 0.001
    if p_value < 0.001:
        p_value_formatted = "< .001"
    else:
        p_value_formatted = round(p_value, 3)

    if p_value < 0.05:
        print("The proportion of women students in {} fields are significantly higher than women workers in {} fields (Z = {}, p = {})"
              .format(l, l, stat_rounded, p_value_formatted))
    else:
        print("The proportion of women students in {} fields are NOT significantly different than women workers in {} fields (Z = {}, p = {})"
              .format(l, l, stat_rounded, p_value_formatted))

The proportion of women students in Architecture and engineering occupations fields are significantly higher than women workers in Architecture and engineering occupations fields (Z = 8.56, p = < .001)
The proportion of women students in Life, physical, and social science occupations fields are significantly higher than women workers in Life, physical, and social science occupations fields (Z = 8.22, p = < .001)
The proportion of women students in Management occupations fields are NOT significantly different than women workers in Management occupations fields (Z = 1.88, p = 0.06)


# Instructions for notebooks from Project Guide:
https://siads593.org/guides/team_project_guidelines/

Please submit “polished” Jupyter *.ipynb notebooks and supporting Python *.py modules (if any). The notebooks must:

Present a coherent, computational narrative of your data manipulation and analysis.
Run without errors from top to bottom.
Run without warnings from top to bottom.
Include a mix of code and Markdown cells.
Include comments in the code, whenever appropriate.
Avoid generating excessive output (i.e., more than 10-15 lines per cell).
Deciding on the number of notebooks to produce is a team decision. You might choose to separate the data acquisition, cleaning and manipulation, and analysis tasks into multiple notebooks in order to narrow the code focus and control for length. A potential downside is code repetition across the notebooks, the likelihood of which increases for the inexperienced and those working under tight time constraints (adoption of code modularization strategies can mitigate this concern). Another option is a single notebook. Opting for this approach places a premium on notebook organization in order to ensure that the reader is able to sensibly traverse an otherwise sprawling set of cells. Regardless of the number of notebooks, the code must be organized in a manner that facilitates comprehension and reuse.

If you have yet to do so consider adopting the following best practices:
(does not copy/paste well with links - go to this page: https://siads593.org/guides/team_project_guidelines/)

To get full points:
* EDA process including cleaning and manipulation steps is well-documented.
* Dependencies are recorded.
* Notebook(s) is/are structured appropriately.
* Cells are well-annotated including use of Markdown text cells to help guide the reader through the notebook.
* Code is modularized.
* Code readability is enhanced by adherence to PEP 8 styling guidelines (https://pep8.org/)
* Visualizations are expressive and effective; marks, labels and color palette are appropriate.
* No warnings or errors are encountered when notebook(s) is/are run.


# Instructions for submission from project guide:
https://siads593.org/guides/team_project_guidelines/

❗ Please submit the team project ZIP archive on or before the due date. Failure to do so will trigger late penalties as described in the Syllabus.

Team project assets must be submitted in a *.zip file. Structure the ZIP archive as follows:

Requirements
Zip archive filename: You must utilize you Coursera project team name as the name of your ZIP file formatted as follows:

1. Zip archive: < team number >-< uniqname >-< uniqname >[-< uniqname >]_[YYYY][fall | winter | sprsum].zip

Example: 00-arwhyte-cteplovs-2023winter.zip

2. Team project report: Your project report must be named the same as your Coursera team name and must be provided as a PDF (e.g., 00-arwhyte-cteplovs-2023winter.pdf). Remember, the project report must be no greater than 11 pages in length. Keep this in mind if you are planning to generate the PDF from a Jupyter notebook. We strongly recommend that you use Google Docs or Slides, Microsoft Word or Powerpoint, or some other word processing package to generate your final PDF.

3. Source code and data: Place all source code files (e.g., Jupyter notebooks, Python scripts, or other code) in a src/ directory in your zip file. Place all data files in a src/data/ directory in your zip file.

❗ Do not submit large data files as part of your zip file but be sure that the teaching team and your classmates can access the data. If a data file is over 10 MB or not available in file form, create a sample file containing the first 100 records. If data was retrieved via an API document the URL endpoint(s) in the relevant Jupyter notebook(s) or Python script(s).

💡 Choose names for your notebook(s), Python *.py script(s), and/or module(s) that convey to the reader each file’s purpose.

ZIP archive structure:

`00-arwhyte-cteplovs-2023winter.zip
src/
    some_notebook.ipynb
    another_notebook.ipynb
    some_script.py
    some_module.py
    ...
    data/
        some_dataset.csv
        another_dataset.json
        sample_data.csv
        ...`

You will also be required to submit your team project report PDF to a gallery tool that will facilitate the sharing of your report with the class. The details of this will be announced during the course.
